In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
base_dir = os.getcwd()

In [3]:
data_filename = "m29_7arcmin_panstarrs_data.csv"
data = pd.read_csv(os.path.join(base_dir, data_filename), sep=';')
useful_columns = ["_r", "RAJ2000", "DEJ2000", "gmag", "e_gmag", "rmag", "e_rmag", "imag", "e_imag", "zmag", "e_zmag"]
data = data[useful_columns]

def to_float(x): 
    try: 
        return float(x)
    except Exception:
        return np.nan 
data = data.applymap(to_float).astype(float)

print(data.describe())

                _r      RAJ2000      DEJ2000         gmag       e_gmag  \
count  6103.000000  6103.000000  6103.000000  2775.000000  1760.000000   
mean      4.713027   305.933606    38.482211    21.518668     0.059655   
std       1.622350     0.074891     0.058212     3.321451     0.068847   
min       0.034900   305.794028    38.370887     8.725000     0.000000   
25%       3.582200   305.872763    38.433972    19.862950     0.009475   
50%       4.991800   305.929423    38.481161    21.577800     0.030750   
75%       6.088200   305.992937    38.528819    23.208000     0.081100   
max       6.999900   306.091144    38.602637    30.090000     0.432200   

              rmag       e_rmag         imag       e_imag         zmag  \
count  3702.000000  3187.000000  5594.000000  5401.000000  5895.000000   
mean     20.065846     0.061443    19.683335     0.050423    19.009028   
std       2.377252     0.069253     1.994553     0.058106     1.806394   
min       7.983500     0.000000     8

In [4]:
data.dropna(subset=["gmag", "rmag", "imag", "zmag"], inplace=True)
gmags = data["gmag"]
print(data.describe())

                _r      RAJ2000      DEJ2000         gmag       e_gmag  \
count  2641.000000  2641.000000  2641.000000  2641.000000  1629.000000   
mean      4.725604   305.935539    38.483476    21.623468     0.055435   
std       1.592680     0.075223     0.058280     3.276687     0.065481   
min       0.034900   305.794028    38.371720     8.725000     0.000000   
25%       3.592300   305.873864    38.435752    19.892500     0.009100   
50%       4.972200   305.929754    38.482309    21.618000     0.028400   
75%       6.075000   305.994990    38.529979    23.485000     0.070900   
max       6.999800   306.091144    38.602637    30.090000     0.432200   

              rmag       e_rmag         imag       e_imag         zmag  \
count  2641.000000  2131.000000  2641.000000  2451.000000  2641.000000   
mean     19.719195     0.036523    18.562242     0.022226    17.803363   
std       2.478741     0.049236     2.039345     0.034142     1.803760   
min       8.338800     0.000000     8

In [11]:
%matplotlib qt
fig = plt.figure()
ax = plt.axes(projection="3d")
gmags = data["gmag"]
rmags = data["rmag"]
imags = data["imag"]
zmags = data["zmag"]

# ax.scatter3D(gmags, rmags - gmags, imags - gmags, s=1, color='red')
# ax.set_xlabel("G mag")
# ax.set_ylabel("R - G")
# ax.set_zlabel("I - G")
# plt.show()
def ugriz_to_ubvri(ugriz):
    u, g, r, i, z = ugriz
    u = np.array(u)
    g = np.array(g)
    r = np.array(r)
    i = np.array(i)
    z = np.array(z)

    V = g - 0.59*(g - r) - 0.01
    B = g + 0.39*(g - r) + 0.21
    R = V - (1.09*(r - i) + 0.22)
    print(f"u: {len(u)}, g: {len(g)}, B: {len(B)}")
    U = 0.77*(u - g) - 0.88 + B
    print(len(U))

    I = R - (r - i + 0.21)
    return (U, B, V, R, I)

umags = pd.Series([0 for _ in range(len(gmags))])
data["U"], data["B"], data["V"], data["R"], data["I"] = ugriz_to_ubvri((umags, gmags, rmags, imags, zmags))



u: 2641, g: 2641, B: 2641
2641


count    2641.000000
mean       17.641915
std         1.994452
min         4.968519
25%        16.699079
50%        17.815018
75%        18.753120
max        25.865894
Name: I, dtype: float64

In [5]:
times = [6.0 + i*0.1 for i in range(31)]

filenames = [f"isochrone_{10*t:.0f}.dat" for t in times]
isochrone_cols = ["M_ini", "MV", "U-B", "B-V", "V-R", "V-I"]
isochrones = {times[i]: pd.read_csv(os.path.join("..", "isochrones", filenames[i]), delim_whitespace=True)[isochrone_cols] for i in range(len(filenames))}


In [6]:

%matplotlib qt
fig = plt.figure()
ax = plt.axes(projection="3d")
vmags = isochrones[9.0]["MV"] 
bmags = isochrones[9.0]["B-V"] + vmags
rmags = vmags - isochrones[9.0]["V-R"]

ax.scatter3D(vmags, bmags - vmags, rmags - vmags, s=1, color='red')
ax.set_xlabel("G mag")
ax.set_ylabel("R - G")
ax.set_zlabel("I - G")
plt.show()